In [ ]:
# =============================
# Step 0: Import libraries
# =============================
import itertools
import numpy as np
import pandas as pd
import nashpy as nash

# =============================
# Step 1: Define types, actions, probabilities
# =============================
S = 100  # total resource cap (baseline)

types_A = [30, 50]   # A's possible minimum types
types_B = [40, 60]   # B's possible minimum types
pA = [0.5, 0.5]      # distribution over A's types
pB = [0.5, 0.5]      # distribution over B's types

actions = [0, 25, 50, 75, 100]  # possible demands

# =============================
# Step 2: Enumerate pure strategy mappings
# =============================
strategies_A = list(itertools.product(actions, repeat=len(types_A)))
strategies_B = list(itertools.product(actions, repeat=len(types_B)))

labels_A = [f"tA={types_A[0]}→{s[0]}, tA={types_A[1]}→{s[1]}" for s in strategies_A]
labels_B = [f"tB={types_B[0]}→{s[0]}, tB={types_B[1]}→{s[1]}" for s in strategies_B]

nA, nB = len(strategies_A), len(strategies_B)
print("Number of pure strategies (A,B):", nA, nB)

# =============================
# Step 3: Build payoff matrices
# =============================
PayoffA = np.zeros((len(strategies_A), len(strategies_B)))
PayoffB = np.zeros((len(strategies_A), len(strategies_B)))

for i, stratA in enumerate(strategies_A):
    for j, stratB in enumerate(strategies_B):
        expected_A, expected_B = 0, 0
        for tAi, tA in enumerate(types_A):
            for tBi, tB in enumerate(types_B):
                prob = pA[tAi] * pB[tBi]
                dA = stratA[tAi]
                dB = stratB[tBi]
                if dA + dB <= S:
                    expected_A += prob * dA
                    expected_B += prob * dB
        PayoffA[i, j] = expected_A
        PayoffB[i, j] = expected_B

# =============================
# Step 4: Display payoff matrices (Figure 1)
# =============================
df_A = pd.DataFrame(PayoffA, index=labels_A, columns=labels_B)
df_B = pd.DataFrame(PayoffB, index=labels_A, columns=labels_B)

print("=== Payoff Matrix for Player A (Figure 1) ===")
display(df_A)
print("=== Payoff Matrix for Player B (Figure 1) ===")
display(df_B)

# =============================
# Step 5: Exhaustive pure-strategy Bayesian Nash equilibrium search
# =============================
pure_bnes = []
for i in range(nA):
    for j in range(nB):
        Ai_best = all(PayoffA[i, j] >= PayoffA[k, j] - 1e-9 for k in range(nA))
        Bj_best = all(PayoffB[i, j] >= PayoffB[i, l] - 1e-9 for l in range(nB))
        if Ai_best and Bj_best:
            pure_bnes.append((i, j, PayoffA[i, j], PayoffB[i, j]))

print("\n=== Pure-strategy Bayesian Nash equilibria ===")
if not pure_bnes:
    print("No pure-strategy BNE found.")
else:
    for idx, (i, j, uA, uB) in enumerate(pure_bnes, 1):
        print(f"Pure BNE #{idx}: A={labels_A[i]}  B={labels_B[j]}  E[U]=(A:{uA:.6f}, B:{uB:.6f})")

# =============================
# Step 6: Mixed equilibria via perturbed support_enumeration
# =============================
def find_mixed_equilibria_with_perturbations(PayoffA, PayoffB, tries=8, eps=1e-8):
    found = {}
    for t in range(tries):
        rng = np.random.RandomState(seed=1234 + t)
        perturbA = rng.normal(size=PayoffA.shape) * eps
        perturbB = rng.normal(size=PayoffB.shape) * eps
        A_pert = PayoffA + perturbA
        B_pert = PayoffB + perturbB
        game = nash.Game(A_pert, B_pert)
        try:
            for eq in game.support_enumeration():
                sigmaA = np.array(eq[0], dtype=float)
                sigmaB = np.array(eq[1], dtype=float)
                sigmaA[sigmaA < 1e-12] = 0.0
                sigmaB[sigmaB < 1e-12] = 0.0
                key = (tuple(np.round(sigmaA, 8)), tuple(np.round(sigmaB, 8)))
                found[key] = (sigmaA, sigmaB)
        except Exception:
            pass
    equilibria = [(v[0], v[1]) for v in found.values()]
    return equilibria

equilibria = find_mixed_equilibria_with_perturbations(PayoffA, PayoffB, tries=10, eps=1e-8)

def pretty_print_eq_list(equilibria, labels_A, labels_B, PayoffA, PayoffB, maxshow=30):
    if not equilibria:
        print(" No mixed equilibria found by this method.")
        return
    for idx, (sigmaA, sigmaB) in enumerate(equilibria, 1):
        expA = float(sigmaA @ PayoffA @ sigmaB)
        expB = float(sigmaA @ PayoffB @ sigmaB)
        suppA = [(i, float(p)) for i, p in enumerate(sigmaA) if p > 1e-8]
        suppB = [(i, float(p)) for i, p in enumerate(sigmaB) if p > 1e-8]
        print(f"\nMixed Eq #{idx}: E[U]=(A:{expA:.6f}, B:{expB:.6f})  | support sizes: A={len(suppA)}, B={len(suppB)}")
        print(" A support (index, prob, label) :")
        for i, p in suppA:
            print(f"   - ({i}) prob={p:.6f}  label={labels_A[i]}")
        print(" B support (index, prob, label) :")
        for j, p in suppB:
            print(f"   - ({j}) prob={p:.6f}  label={labels_B[j]}")
        if idx >= maxshow:
            print(" ... (truncated output) ...")
            break

print("\n=== Mixed equilibria (perturbed support enumeration) ===")
pretty_print_eq_list(equilibria, labels_A, labels_B, PayoffA, PayoffB)

# =============================
# Step 7: Lemke-Howson on original game
# =============================
print("\n=== Lemke-Howson equilibria ===")
game_orig = nash.Game(PayoffA, PayoffB)
found_lh = {}
for start in range(1, min(nA, nB) + 1):
    try:
        sa, sb = game_orig.lemke_howson(start)
        sa = np.array(sa, dtype=float); sb = np.array(sb, dtype=float)
        sa[sa < 1e-12] = 0.0; sb[sb < 1e-12] = 0.0
        key = (tuple(np.round(sa, 8)), tuple(np.round(sb, 8)))
        found_lh[key] = (sa, sb)
    except Exception:
        pass

found_lh = [(v[0], v[1]) for v in found_lh.values()]
pretty_print_eq_list(found_lh, labels_A, labels_B, PayoffA, PayoffB)

# =============================
# Step 8: Summary
# =============================
print("\n=== SUMMARY ===")
print("Pure BNE count:", len(pure_bnes))
print("Mixed (perturbed support enumeration) approx count:", len(equilibria))
print("Lemke-Howson approx count:", len(found_lh))


Number of pure strategies (A,B): 25 25
=== Payoff Matrix for Player A (Figure 1) ===


,"tB=40→0, tB=60→0","tB=40→0, tB=60→25","tB=40→0, tB=60→50","tB=40→0, tB=60→75","tB=40→0, tB=60→100","tB=40→25, tB=60→0","tB=40→25, tB=60→25","tB=40→25, tB=60→50","tB=40→25, tB=60→75","tB=40→25, tB=60→100",...,"tB=40→75, tB=60→0","tB=40→75, tB=60→25","tB=40→75, tB=60→50","tB=40→75, tB=60→75","tB=40→75, tB=60→100","tB=40→100, tB=60→0","tB=40→100, tB=60→25","tB=40→100, tB=60→50","tB=40→100, tB=60→75","tB=40→100, tB=60→100"
"tA=30→0, tA=50→0",0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0
"tA=30→0, tA=50→25",12.5,12.5,12.50,12.50,6.25,12.5,12.5,12.50,12.50,6.25,...,12.50,12.50,12.5,12.5,6.25,6.25,6.25,6.25,6.25,0.0
"tA=30→0, tA=50→50",25.0,25.0,25.00,12.50,12.50,25.0,25.0,25.00,12.50,12.50,...,12.50,12.50,12.5,0.0,0.00,12.50,12.50,12.50,0.00,0.0
"tA=30→0, tA=50→75",37.5,37.5,18.75,18.75,18.75,37.5,37.5,18.75,18.75,18.75,...,18.75,18.75,0.0,0.0,0.00,18.75,18.75,0.00,0.00,0.0
"tA=30→0, tA=50→100",50.0,25.0,25.00,25.00,25.00,25.0,0.0,0.00,0.00,0.00,...,25.00,0.00,0.0,0.0,0.00,25.00,0.00,0.00,0.00,0.0
"tA=30→25, tA=50→0",12.5,12.5,12.50,12.50,6.25,12.5,12.5,12.50,12.50,6.25,...,12.50,12.50,12.5,12.5,6.25,6.25,6.25,6.25,6.25,0.0
"tA=30→25, tA=50→25",25.0,25.0,25.00,25.00,12.50,25.0,25.0,25.00,25.00,12.50,...,25.00,25.00,25.0,25.0,12.50,12.50,12.50,12.50,12.50,0.0
"tA=30→25, tA=50→50",37.5,37.5,37.50,25.00,18.75,37.5,37.5,37.50,25.00,18.75,...,25.00,25.00,25.0,12.5,6.25,18.75,18.75,18.75,6.25,0.0
"tA=30→25, tA=50→75",50.0,50.0,31.25,31.25,25.00,50.0,50.0,31.25,31.25,25.00,...,31.25,31.25,12.5,12.5,6.25,25.00,25.00,6.25,6.25,0.0
"tA=30→25, tA=50→100",62.5,37.5,37.50,37.50,31.25,37.5,12.5,12.50,12.50,6.25,...,37.50,12.50,12.5,12.5,6.25,31.25,6.25,6.25,6.25,0.0


=== Payoff Matrix for Player B (Figure 1) ===


,"tB=40→0, tB=60→0","tB=40→0, tB=60→25","tB=40→0, tB=60→50","tB=40→0, tB=60→75","tB=40→0, tB=60→100","tB=40→25, tB=60→0","tB=40→25, tB=60→25","tB=40→25, tB=60→50","tB=40→25, tB=60→75","tB=40→25, tB=60→100",...,"tB=40→75, tB=60→0","tB=40→75, tB=60→25","tB=40→75, tB=60→50","tB=40→75, tB=60→75","tB=40→75, tB=60→100","tB=40→100, tB=60→0","tB=40→100, tB=60→25","tB=40→100, tB=60→50","tB=40→100, tB=60→75","tB=40→100, tB=60→100"
"tA=30→0, tA=50→0",0.0,12.50,25.0,37.50,50.0,12.50,25.0,37.50,50.00,62.50,...,37.50,50.00,62.50,75.0,87.50,50.0,62.50,75.0,87.50,100.0
"tA=30→0, tA=50→25",0.0,12.50,25.0,37.50,25.0,12.50,25.0,37.50,50.00,37.50,...,37.50,50.00,62.50,75.0,62.50,25.0,37.50,50.0,62.50,50.0
"tA=30→0, tA=50→50",0.0,12.50,25.0,18.75,25.0,12.50,25.0,37.50,31.25,37.50,...,18.75,31.25,43.75,37.5,43.75,25.0,37.50,50.0,43.75,50.0
"tA=30→0, tA=50→75",0.0,12.50,12.5,18.75,25.0,12.50,25.0,25.00,31.25,37.50,...,18.75,31.25,31.25,37.5,43.75,25.0,37.50,37.5,43.75,50.0
"tA=30→0, tA=50→100",0.0,6.25,12.5,18.75,25.0,6.25,12.5,18.75,25.00,31.25,...,18.75,25.00,31.25,37.5,43.75,25.0,31.25,37.5,43.75,50.0
"tA=30→25, tA=50→0",0.0,12.50,25.0,37.50,25.0,12.50,25.0,37.50,50.00,37.50,...,37.50,50.00,62.50,75.0,62.50,25.0,37.50,50.0,62.50,50.0
"tA=30→25, tA=50→25",0.0,12.50,25.0,37.50,0.0,12.50,25.0,37.50,50.00,12.50,...,37.50,50.00,62.50,75.0,37.50,0.0,12.50,25.0,37.50,0.0
"tA=30→25, tA=50→50",0.0,12.50,25.0,18.75,0.0,12.50,25.0,37.50,31.25,12.50,...,18.75,31.25,43.75,37.5,18.75,0.0,12.50,25.0,18.75,0.0
"tA=30→25, tA=50→75",0.0,12.50,12.5,18.75,0.0,12.50,25.0,25.00,31.25,12.50,...,18.75,31.25,31.25,37.5,18.75,0.0,12.50,12.5,18.75,0.0
"tA=30→25, tA=50→100",0.0,6.25,12.5,18.75,0.0,6.25,12.5,18.75,25.00,6.25,...,18.75,25.00,31.25,37.5,18.75,0.0,6.25,12.5,18.75,0.0



=== Pure-strategy Bayesian Nash equilibria ===
Pure BNE #1: A=tA=30→0, tA=50→0  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:100.000000)
Pure BNE #2: A=tA=30→0, tA=50→50  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:50.000000)
Pure BNE #3: A=tA=30→0, tA=50→75  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:50.000000)
Pure BNE #4: A=tA=30→0, tA=50→100  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:50.000000)
Pure BNE #5: A=tA=30→25, tA=50→25  B=tB=40→75, tB=60→75  E[U]=(A:25.000000, B:75.000000)
Pure BNE #6: A=tA=30→50, tA=50→0  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:50.000000)
Pure BNE #7: A=tA=30→50, tA=50→50  B=tB=40→50, tB=60→50  E[U]=(A:50.000000, B:50.000000)
Pure BNE #8: A=tA=30→75, tA=50→0  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:50.000000)
Pure BNE #9: A=tA=30→75, tA=50→75  B=tB=40→25, tB=60→25  E[U]=(A:75.000000, B:25.000000)
Pure BNE #10: A=tA=30→100, tA=50→0  B=tB=40→100, tB=60→100  E[U]=(A:0.000000, B:50.000000)
Pure BNE #11: A=tA=30→100, tA=50→100  B=tB=40→0, tB=60→0  E